# Under Construction 🚧

In [ ]:

import datasets
import seaborn as sns
sns.set_style("whitegrid")
import matplotlib.pyplot as plt
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from torch.nn import Linear, CrossEntropyLoss
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import Muon, AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
import sys
from pathlib import Path
from torch.utils.tensorboard import SummaryWriter
sys.path.append(str(Path.cwd().parent))
from transformers import DataCollatorWithPadding
from utils import remove_extra_brackets, CLASSIFICATION_PROMPT
from peft import LoraConfig, get_peft_model
import loralib as lora

In [2]:
BATCH_SIZE = 4

In [3]:
writer = SummaryWriter("./logs/qwen_llm_finetune")

In [4]:
# Load multiple CSV files
df = datasets.load_dataset('csv', data_files={
    'train': '../data/train.csv',
    'test': '../data/test.csv'
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B-Instruct-2507")
print(f"Model max length is {tokenizer.model_max_length} characters.")
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-4B-Instruct-2507", device_map="cuda"
)
model.lm_head = Linear(model.config.hidden_size, 3, bias=False).to("cuda")
model

`torch_dtype` is deprecated! Use `dtype` instead!


Model max length is 131072 characters.


Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layer

In [ ]:
# add LoRA adaptor
model = get_peft_model(model, lora_config)
lora.mark_only_lora_as_trainable(model)

# Enable gradient checkpointing on raw pytorch
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.config.use_cache = False

model.print_trainable_parameters()

/venv/main/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:916: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


## Fix Dataset

In [8]:

def fix_dataset(row):
    cleaned_prompt = remove_extra_brackets(row['prompt'])
    cleaned_response_a = remove_extra_brackets(row['response_a'])
    cleaned_response_b = remove_extra_brackets(row['response_b'])
    
    full_prompt = CLASSIFICATION_PROMPT.format(
        prompt=cleaned_prompt,
        response_a=cleaned_response_a,
        response_b=cleaned_response_b
    )
    
    tokenized = tokenizer(
        full_prompt
    )
    
    winner = [row['winner_model_a'], row['winner_model_b'], row['winner_tie']]
    
    return {
        **tokenized,
        "winner": winner,
        "length": len(tokenized["input_ids"])
    }

In [9]:
df = df.map(fix_dataset, batched=False).remove_columns(['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b','winner_model_a', 'winner_model_b', 'winner_tie'])

In [10]:
df = df.filter(lambda batch: np.array(batch["length"]) <= 12000, batched=True).remove_columns(["length"])

In [11]:
train_val_split = df['train'].train_test_split(test_size=0.05, seed=42)
df['train'] = train_val_split['train']
df['validation'] = train_val_split['test']
df = df.with_format("torch")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)
train_dataloader = DataLoader(df["train"], batch_size=BATCH_SIZE, shuffle=True, collate_fn=data_collator)
val_dataloader = DataLoader(df["validation"], batch_size=BATCH_SIZE, shuffle=False, collate_fn=data_collator)

# Experiment

In [12]:
# data = next(iter(train_dataloader))
# data["input_ids"].shape
# data["input_ids"] = data["input_ids"].to("cuda")
# with torch.no_grad():
#     with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
#         outputs = model(data["input_ids"]).logits
#         sums = data["attention_mask"].sum(dim=1) - 1
# preds = outputs[torch.arange(2), sums, :]
# torch.argmax(preds, dim=1)
# torch.argmax(data['winner'], dim=1)

# Training

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
# optimizer = bnb.optim.Adam(model.parameters(), lr=1e-4, optim_bits=32)
EPOCHS = 10
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
loss_fn = CrossEntropyLoss()
GRADIENT_ACCUMULATION_STEPS = 64

torch.set_float32_matmul_precision("medium")

grad_steps_corrects = 0
grad_steps_count = 0

train_size = len(train_dataloader)

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    total_correct = 0
    total_count = 0
    optimizer.zero_grad()

    train_bar = tqdm(
        train_dataloader, desc=f"Epoch {epoch + 1}/{EPOCHS}", leave=True, position=0
    )
    validation_bar = tqdm(
        val_dataloader,
        desc=f"Validation {epoch + 1}/{EPOCHS}",
        leave=False,
        position=0,
    )
    for step, data in enumerate(train_bar):
        data = {key: value.to("cuda") for key, value in data.items()}

        with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
            outputs = outputs = model(data["input_ids"]).logits
            last_row_positions = data["attention_mask"].sum(dim=1) - 1
            preds = outputs[torch.arange(BATCH_SIZE), last_row_positions, :]
            with torch.no_grad():
                _, predicted = torch.max(preds, 1)
                _, true_labels = torch.max(data["winner"], 1)
                examples_count = data["input_ids"].size(0)
                correct_count = (predicted == true_labels).sum().item()
                grad_steps_count += examples_count
                total_count += examples_count
                total_correct += correct_count
                grad_steps_corrects += correct_count
                if (step + 1) % (GRADIENT_ACCUMULATION_STEPS // 4) == 0:
                    train_bar.set_postfix(
                        {
                            "Prediction": f"{predicted.cpu().tolist()} | {true_labels.cpu().tolist()}"
                        }
                    )

            loss = loss_fn(preds, true_labels)

        (loss / GRADIENT_ACCUMULATION_STEPS).backward()

        total_loss += loss.item()

        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            accuracy = 100 * (grad_steps_corrects / grad_steps_count)
            grad_steps_corrects = 0
            grad_steps_count = 0

            train_bar.set_postfix({"accuracy": f"{(accuracy):.2f}%"})
            # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            optimizer.zero_grad()
            writer.add_scalar(
                "Accuracy/train", accuracy, (epoch * train_size) + (step + 1)
            )

        torch.cuda.empty_cache()

        if step % 16000 == 0 and step != 0:
            model.eval()
            correct = 0
            total = 0
            for data in validation_bar:
                with torch.no_grad():
                    with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
                        data = {
                            key: value.to("cuda") for key, value in data.items()
                        }
                        outputs = model(data["input_ids"]).logits
                        last_row_positions = data["attention_mask"].sum(dim=1) - 1
                        preds = outputs[
                            torch.arange(BATCH_SIZE), last_row_positions, :
                        ]
                        _, predicted = torch.max(preds, 1)
                        _, true_labels = torch.max(data["winner"], 1)
                        total += true_labels.size(0)
                        correct += (predicted == true_labels).sum().item()

            accuracy = 100 * (correct / total)
            print(f"Validation Accuracy: {accuracy:.2f}%")
            writer.add_scalar(
                "Accuracy/validation", accuracy, (epoch * train_size) + (step + 1)
            )
            model.train()

    scheduler.step()
    avg_loss = total_loss / len(train_dataloader)
    print(
        f"Epoch {epoch + 1}/{EPOCHS}, Avg Loss: {avg_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.2e}"
    )
    print(
        f"Epoch {epoch + 1}/{EPOCHS}, Training Accuracy: {100 * (total_correct / total_count):.2f}%"
    )

Validation 1/10:   0%|          | 0/719 [00:00<?, ?it/s]

Epoch 1/10:   0%|          | 30/13644 [00:24<3:03:34,  1.24it/s, Prediction=[1, 1, 1, 1] | [2, 2, 1, 0]]


OutOfMemoryError: CUDA out of memory. Tried to allocate 82.00 MiB. GPU 0 has a total capacity of 23.57 GiB of which 51.12 MiB is free. Process 3191043 has 23.51 GiB memory in use. Of the allocated memory 23.06 GiB is allocated by PyTorch, and 143.81 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)